<h1>1.3 - Weight Decay</h1>

Let's continue from our multivariate linear regression. 
Now let's incorporate the $\ell_2$ regularization ($L_{2}$) into our model.

In [1]:
import torch
from torch import nn

from platform import python_version
python_version(), torch.__version__

('3.12.6', '2.5.1+cu124')

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device

'cuda'

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
def add_to_class(Class):  
    """Register functions as methods in created class."""
    def wrapper(obj): setattr(Class, obj.__name__, obj)
    return wrapper

# Dataset

## create dataset

$$
\begin{align*}
\mathbf{X} &\in \mathbb{R}^{m \times n} \\
\mathbf{Y} &\in \mathbb{R}^{m \times n_{1}}
\end{align*}
$$

In [5]:
from sklearn.datasets import make_regression
import random

M: int = 10_100 # number of samples
N: int = 6 # number of input features
NO: int = 3 # number of output features

X, Y = make_regression(
    n_samples=M, 
    n_features=N, 
    n_targets=NO, 
    n_informative=N - 1,
    bias=random.random(),
    noise=1
)

print(X.shape)
print(Y.shape)

(10100, 6)
(10100, 3)


## split dataset

In [6]:
x_train = torch.tensor(X[:100], device=device)
y_train = torch.tensor(Y[:100], device=device)
x_train.shape, y_train.shape

(torch.Size([100, 6]), torch.Size([100, 3]))

In [7]:
x_valid = torch.tensor(X[100:], device=device)
y_valid = torch.tensor(Y[100:], device=device)
x_valid.shape, y_valid.shape

(torch.Size([10000, 6]), torch.Size([10000, 3]))

## delete raw dataset

In [8]:
del X
del Y

# Scratch model

The only thing we are going to modify is the way in which the model weights are updated. 
The rest, such as parameter initialization and model training, remain unchanged.

## Linear Regression model

In [9]:
class LinearRegression:
    def __init__(self, n_features: int, out_features: int, lambd: float):
        self.w = torch.randn(n_features, out_features, device=device)
        self.b = torch.randn(out_features, device=device)
        self.lambd = lambd

    def copy_params(self, torch_layer: torch.nn.modules.linear.Linear):
        """
        Copy the parameters from a module.linear to this model.

        Args:
            torch_layer: Pytorch module from which to copy the parameters.
        """
        self.b.copy_(torch_layer.bias.detach().clone())
        self.w.copy_(torch_layer.weight.T.detach().clone())

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """
        Predict the output for input x

        Args:
            x: Input tensor of shape (n_samples, n_features).

        Returns:
            y_pred: Predicted output tensor of shape (n_samples, out_features).
        """
        return torch.matmul(x, self.w) + self.b

    def mse_loss(self, y_true: torch.Tensor, y_pred: torch.Tensor):
        """
        MSE loss function between target y_true and y_pred.

        Args:
            y_true: Target tensor of shape (n_samples, out_features).
            y_pred: Predicted tensor of shape (n_samples, out_features).

        Returns:
            loss: MSE loss between predictions and true values.
        """
        return ((y_pred - y_true)**2).mean().item()

    def evaluate(self, x: torch.Tensor, y_true: torch.Tensor):
        """
        Evaluate the model on input x and target y_true using MSE.

        Args:
            x: Input tensor of shape (n_samples, n_features).
            y_true: Target tensor of shape (n_samples, out_features).

        Returns:
            loss: MSE loss between predictions and true values.
        """
        y_pred = self.predict(x)
        return self.mse_loss(y_true, y_pred)

    def fit(self, x_train: torch.Tensor, y_train: torch.Tensor, 
        epochs: int, lr: float, batch_size: int, 
        x_valid: torch.Tensor, y_valid: torch.Tensor):
        """
        Fit the model using gradient descent.
        
        Args:
            x_train: Input tensor of shape (n_samples, n_features).
            y_train: Target tensor of shape (n_samples,).
            epochs: Number of epochs to fit.
            lr: learning rate.
            batch_size: Int number of batch.
            x_valid: Input tensor of shape (n_valid_samples, n_features).
            y_valid: Target tensor of shape (n_valid_samples,)
        """
        for epoch in range(epochs):
            loss = []
            for batch in range(0, len(y_train), batch_size):
                end_batch = batch + batch_size

                y_pred = self.predict(x_train[batch:end_batch])

                loss.append(self.mse_loss(
                    y_train[batch:end_batch], 
                    y_pred
                ))

                self.update(
                    x_train[batch:end_batch], 
                    y_train[batch:end_batch], 
                    y_pred, 
                    lr
                )

            loss = round(sum(loss) / len(loss), 4)
            loss_v = round(self.evaluate(x_valid, y_valid), 4)
            print(f'epoch: {epoch} - MSE: {loss} - MSE_v: {loss_v}')

## Parameters update

### objective function

Now instead of training the model with the gradient of the loss function, 
we are going to use the objective function $J$. 
Typically our objective function is as follows.

$$
J(\hat{\mathbf{Y}}, \mathbf{\theta}) = 
L(\hat{\mathbf{Y}}) + \text{regularization}
$$
where $\mathbf{\theta}$ is an arbitrary parameter.

**Note**: Do not use the objective function to evaluate the model.

### L2 regularization

As a regularization technique, we will use weight decay, commonly $\ell_2$ or $L_{2}$.

$$
\ell_2(\mathbf{\theta}) = 
\frac{\lambda}{2} \left\| \mathbf{\theta} \right\|^{2}_{2}
$$
where commonly $\mathbf{\theta} \in \mathbb{R}^{n}$.

But we have $\mathbf{W} \in \mathbb{R}^{n \times n_{1}}$, then we need to do an equivalence operation.

$$
\begin{align*}
\ell_2(\mathbf{W}) &= \frac{\lambda}{2} \sum_{i=1}^{n} \sum_{j=1}^{n_{1}} w_{ij}^{2} \\
&= \frac{\lambda}{2} \text{sum} \left( \mathbf{W}^{2} \right) 
\end{align*}
$$
where ${\mathbf{A}}^2$ is element-wise power or also ${\mathbf{A}}^2 = \mathbf{A} \odot \mathbf{A}$.

### Objective function derivative

$$
\frac{\partial J}{\partial w_{pq}} =
\frac{\partial L}{\partial w_{pq}} +
\frac{\partial \ell_2}{\partial w_{pq}}
$$

$$
\begin{align*}
\frac{\partial \ell_2}{\partial w_{pq}} &=
\frac{\lambda}{2} \sum_{i=1}^{n} \sum_{j=1}^{n_{1}}
\frac{\partial}{\partial w_{pq}} \left( 
    w_{ij}^{2}
\right) \\
&= \lambda w_{pq}
\end{align*}
$$

Because
$$
\frac{\partial w_{ij}}{\partial w_{pq}} = \begin{cases}
    1 & \text{if } i=p, j=q \\
    0 & \text{otherwise}
\end{cases}
$$

In general, for all $p = 1, \ldots, n$ and $q = 1, \ldots, n_{1}$.
$$
\frac{\partial \ell_2}{\partial \mathbf{W}} =
\lambda \mathbf{W}
$$
**Remark**: $\nabla_{\mathbf{W}}\ell_2 \in \mathbb{R}^{n \times n_{1}}$.

$$
\begin{align*}
\frac{\partial J}{\partial \mathbf{W}} &=
{\color{Orange} \frac{\partial L}{\partial \mathbf{W}}} +
{\color{Cyan} \frac{\partial \ell_2}{\partial \mathbf{W}}} \\
&= {\color{Orange} \nabla_{\mathbf{W}}L} + 
{\color{Cyan} \lambda \mathbf{W}}
\end{align*}
$$

In [10]:
@add_to_class(LinearRegression)
def update(self, x: torch.Tensor, y_true: torch.Tensor, y_pred: torch.Tensor, lr: float):
    """
    Update the model parameters with L2 regularization.

    Args:
       x: Input tensor of shape (n_samples, n_features).
       y_true: Target tensor of shape (n_samples, n_features).
       y_pred: Predicted output tensor of shape (n_samples, n_features).
       lr: Learning rate. 
    """
    delta = 2 * (y_pred - y_true) / y_true.numel()
    self.b -= lr * delta.sum(axis=0)
    self.w -= lr * (torch.matmul(x.T, delta) + self.lambd * self.w) # L2 regularization

# Scratch vs Torch.nn

## Torch.nn model

In [11]:
class TorchLinearRegression(nn.Module):
    def __init__(self, n_features, n_out_features):
        super(TorchLinearRegression, self).__init__()
        self.layer = nn.Linear(n_features, n_out_features, device=device)
        self.loss = nn.MSELoss()

    def forward(self, x):
        return self.layer(x)
    
    def evaluate(self, x, y):
        self.eval()
        with torch.no_grad():
            y_pred = self.forward(x)
            return self.loss(y_pred, y).item()
    
    def fit(self, x, y, epochs, lr, batch_size, x_valid, y_valid, weight_decay):
        
        optimizer = torch.optim.SGD([
            {'params': self.layer.weight, 'weight_decay': weight_decay},
            {'params': self.layer.bias, 'weight_decay': 0.0} # it is important to specify the weight decay for the bias.
        ], lr=lr)

        for epoch in range(epochs):
            loss_t = []
            for batch in range(0, len(y), batch_size):
                end_batch = batch + batch_size

                y_pred = self.forward(x[batch:end_batch])
                loss = self.loss(y_pred, y[batch:end_batch])
                loss_t.append(loss.item())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            loss_t = round(sum(loss_t) / len(loss_t), 4)
            loss_v = round(self.evaluate(x_valid, y_valid), 4)
            print(f'epoch: {epoch} - MSE: {loss_t} - MSE_v: {loss_v}')

In [12]:
torch_model = TorchLinearRegression(N, NO)

## scratch model

In [13]:
LAMBD: float = 0.01

model = LinearRegression(N, NO, LAMBD)
model.lambd

0.01

## evals

### porcentual error

In [14]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tools.torch_metrics import torch_mape as mape

### predict

In [15]:
mape(
    model.predict(x_valid),
    torch_model.forward(x_valid)
)

36.2726757107143

### copy parameters

In [16]:
model.copy_params(torch_model.layer)
parameters = (model.b.clone(), model.w.clone())

### predict after copy parameters

In [17]:
mape(
    model.predict(x_valid),
    torch_model.forward(x_valid)
)

0.0

### loss

In [18]:
mape(
    model.evaluate(x_valid, y_valid),
    torch_model.evaluate(x_valid, y_valid)
)

0.0

### train

In [19]:
LR = 0.01 # learning rate
EPOCHS = 16 # number of epochs
BATCH = len(x_train) // 3 # batch size

In [20]:
torch_model.fit(
    x_train, y_train, 
    EPOCHS, LR, BATCH, 
    x_valid, y_valid,
    LAMBD
)

epoch: 0 - MSE: 16528.9 - MSE_v: 14972.9969
epoch: 1 - MSE: 15473.1382 - MSE_v: 14153.2766
epoch: 2 - MSE: 14492.454 - MSE_v: 13385.6783
epoch: 3 - MSE: 13581.1496 - MSE_v: 12666.5176
epoch: 4 - MSE: 12733.9758 - MSE_v: 11992.3881
epoch: 5 - MSE: 11946.0959 - MSE_v: 11360.1401
epoch: 6 - MSE: 11213.0529 - MSE_v: 10766.8599
epoch: 7 - MSE: 10530.7385 - MSE_v: 10209.8521
epoch: 8 - MSE: 9895.3657 - MSE_v: 9686.6219
epoch: 9 - MSE: 9303.4428 - MSE_v: 9194.8599
epoch: 10 - MSE: 8751.7499 - MSE_v: 8732.4273
epoch: 11 - MSE: 8237.3173 - MSE_v: 8297.3432
epoch: 12 - MSE: 7757.4051 - MSE_v: 7887.7719
epoch: 13 - MSE: 7309.4858 - MSE_v: 7502.0121
epoch: 14 - MSE: 6891.2264 - MSE_v: 7138.4862
epoch: 15 - MSE: 6500.4736 - MSE_v: 6795.7309


In [21]:
model.fit(
    x_train, y_train, 
    EPOCHS, LR, BATCH, 
    x_valid, y_valid
)

epoch: 0 - MSE: 16528.9 - MSE_v: 14972.9969
epoch: 1 - MSE: 15473.1382 - MSE_v: 14153.2766
epoch: 2 - MSE: 14492.454 - MSE_v: 13385.6783
epoch: 3 - MSE: 13581.1496 - MSE_v: 12666.5176
epoch: 4 - MSE: 12733.9758 - MSE_v: 11992.3881
epoch: 5 - MSE: 11946.0959 - MSE_v: 11360.1401
epoch: 6 - MSE: 11213.0529 - MSE_v: 10766.8599
epoch: 7 - MSE: 10530.7385 - MSE_v: 10209.8521
epoch: 8 - MSE: 9895.3657 - MSE_v: 9686.6219
epoch: 9 - MSE: 9303.4428 - MSE_v: 9194.8599
epoch: 10 - MSE: 8751.7499 - MSE_v: 8732.4273
epoch: 11 - MSE: 8237.3173 - MSE_v: 8297.3432
epoch: 12 - MSE: 7757.4051 - MSE_v: 7887.7719
epoch: 13 - MSE: 7309.4858 - MSE_v: 7502.0121
epoch: 14 - MSE: 6891.2264 - MSE_v: 7138.4862
epoch: 15 - MSE: 6500.4736 - MSE_v: 6795.7309


### predict after training

In [22]:
mape(
    model.predict(x_valid),
    torch_model.forward(x_valid)
)

5.554492395677917e-16

### weight 

In [23]:
mape(
    model.w.clone(),
    torch_model.layer.weight.detach().T
)

8.717837983297495e-17

### bias

In [24]:
mape(
    model.b.clone(),
    torch_model.layer.bias.detach()
)

5.1857799110327954e-17